In [1]:

import datetime
import pandas as pd
import yfinance as yf
import pandas_datareader.data as web
import numpy as np
import scipy as sp
from scipy.optimize import minimize



In [223]:
ap = yf.download('AAPL', start="2023-01-01", end="2023-05-01")
gg = yf.download('GOOG', start="2023-01-01", end="2023-05-01")
ms = yf.download('MSFT', start="2023-01-01", end="2023-05-01")
ts = yf.download('TSLA', start="2023-01-01", end="2023-05-01")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [224]:
stocks = pd.concat([ap['Adj Close'], gg['Adj Close'], ms['Adj Close'], ts['Adj Close']],axis=1)


In [225]:
stocks.columns = ['Apple', 'Google', 'Microsoft', 'Tesla']

In [226]:
## Change in returns over time

returns = stocks/stocks.shift(1)
returns

,Apple,Google,Microsoft,Tesla
Date,,,,
2023-01-03,NaN,NaN,NaN,NaN
2023-01-04,1.010314,0.988963,0.956257,1.051249
2023-01-05,0.989395,0.978131,0.970362,0.970961
2023-01-06,1.036794,1.016019,1.011785,1.024651
2023-01-09,1.004089,1.007260,1.009736,1.059349
...,...,...,...,...
2023-04-24,1.001879,1.008214,0.986037,0.984674
2023-04-25,0.990564,0.979678,0.977464,0.988434
2023-04-26,0.999939,0.998470,1.072435,0.956930


In [227]:
## Find mean of returns

returns.mean()

Apple        1.003936
Google       1.002617
Microsoft    1.003340
Tesla        1.006018
dtype: float64

In [228]:
logReturns = np.log(returns)
logReturns

,Apple,Google,Microsoft,Tesla
Date,,,,
2023-01-03,NaN,NaN,NaN,NaN
2023-01-04,0.010261,-0.011098,-0.044729,0.049979
2023-01-05,-0.010661,-0.022112,-0.030086,-0.029469
2023-01-06,0.036133,0.015892,0.011716,0.024352
2023-01-09,0.004081,0.007233,0.009689,0.057655
...,...,...,...,...
2023-04-24,0.001877,0.008181,-0.014061,-0.015445
2023-04-25,-0.009480,-0.020531,-0.022794,-0.011633
2023-04-26,-0.000061,-0.001531,0.069932,-0.044025


In [229]:
pbar = logReturns.mean()
sigma = logReturns.cov()
pbar
sigma

,Apple,Google,Microsoft,Tesla
Apple,0.000195,0.000218,0.000157,0.000267
Google,0.000218,0.000542,0.000292,0.000288
Microsoft,0.000157,0.000292,0.000390,0.000218
Tesla,0.000267,0.000288,0.000218,0.001549


In [230]:
rMin = 0.02
def riskFunction(w):
    return np.dot(w.T,np.dot(sigma,w))

w0 = [0.25, 0.25, 0.25, 0.25]
bounds = ((0,1), (0,1), (0,1), (0,1))

def checkMinimumRertun(w):
    RHS = rMin - np.sum(pbar * w)
    return RHS
def checkSumToOne(w):
    return np.sum(w) - 1 

constraints = ({'type': 'eq', 'fun': checkMinimumRertun},{'type': 'eq', 'fun': checkSumToOne})
w_opt = minimize(riskFunction, w0, method='SLSQP', bounds=bounds, constraints=constraints)

In [235]:
w_opt

 message: Positive directional derivative for linesearch
 success: False
  status: 8
     fun: 0.0015485991983561684
       x: [ 3.140e-14  7.186e-15  6.323e-16  1.000e+00]
     nit: 23
     jac: [ 5.342e-04  5.762e-04  4.358e-04  3.097e-03]
    nfev: 247
    njev: 19

In [236]:
w_scipy = w_opt.x
w_scipy

array([3.14044574e-14, 7.18629216e-15, 6.32322982e-16, 1.00000000e+00])

In [237]:
risk_scipy = riskFunction(w_scipy)
risk_scipy

0.0015485991983561684

In [238]:
np.sum(pbar*w_scipy)


0.005233727025943622

In [240]:
def markowitz(rMin,sigma,pbar):
    N = len(sigma)
    o = np.ones(N)
    sigmaInv = np.linalg.inv(sigma)
    a = np.dot(pbar.T, np.dot(sigmaInv,pbar))
    b = np.dot(pbar.T,np.dot(sigmaInv,o))
    c = np.dot(o.T,np.dot(sigmaInv,o))
    return (1/(a*c - b**2)) * np.dot(sigmaInv,((c*rMin - b)*pbar + (a-b*rMin)*o))

In [241]:
analysis = markowitz(rMin,sigma,pbar)
analysis

array([ 8.41537723, -8.62050435, -0.79797783,  2.00310495])